In [5]:
import pandas as pd
import numpy as np

# Setting the seed for reproducibility
np.random.seed(42)

# Generate synthetic data
years = np.arange(2013, 2024)  # Historical years
products = ['Smartphone', 'Laptop', 'Television', 'Headphones', 'Washing Machine', 'Refrigerator','watch']
platforms = ['Amazon', 'Flipkart']

data = {
    'Year': np.repeat(years, len(products) * len(platforms)),
    'Product': np.tile(products, len(years) * len(platforms)),
    'Platform': np.repeat(platforms, len(years) * len(products)),
    'Base_Price': np.random.uniform(10000, 100000, len(years) * len(products) * len(platforms)),
    'Discount_Percentage': np.random.uniform(10, 70, len(years) * len(products) * len(platforms)),
}

discount_data = pd.DataFrame(data)
discount_data['Discounted_Price'] = discount_data['Base_Price'] * (1 - discount_data['Discount_Percentage'] / 100)
discount_data.to_csv('synthetic_discount_data_amazon_flipkart.csv', index=False)
print(discount_data)


     Year          Product  Platform    Base_Price  Discount_Percentage  \
0    2013       Smartphone    Amazon  43708.610696            69.139027   
1    2013           Laptop    Amazon  95564.287577            24.523316   
2    2013       Television    Amazon  75879.454763            50.328133   
3    2013       Headphones    Amazon  63879.263578            55.697177   
4    2013  Washing Machine    Amazon  24041.677640            24.258253   
..    ...              ...       ...           ...                  ...   
149  2023       Television  Flipkart  35078.181781            48.245794   
150  2023       Headphones  Flipkart  91743.929737            53.565480   
151  2023  Washing Machine  Flipkart  31560.570160            68.551125   
152  2023     Refrigerator  Flipkart  23040.538488            40.978021   
153  2023            watch  Flipkart  54050.748425            29.377388   

     Discounted_Price  
0        13488.902438  
1        72128.755073  
2        37690.741968  
3  

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Load the synthetic dataset
discount_data = pd.read_csv('synthetic_discount_data_amazon_flipkart.csv')
print(discount_data.head())

# Encode categorical variables
discount_data['Product'] = pd.Categorical(discount_data['Product']).codes
discount_data['Platform'] = pd.Categorical(discount_data['Platform']).codes

# Split the dataset by platform
amazon_data = discount_data[discount_data['Platform'] == 0]
flipkart_data = discount_data[discount_data['Platform'] == 1]

# Function to train and predict
def train_predict(data):
    X = data[['Year', 'Product', 'Base_Price']]
    y = data['Discount_Percentage']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    rf_model = RandomForestRegressor(random_state=42)
    rf_model.fit(X_train, y_train)
    
    y_pred = rf_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f'Mean Absolute Error: {mae}')
    return rf_model

# Train models for Amazon and Flipkart
print("Amazon Model:")
amazon_model = train_predict(amazon_data)

print("\nFlipkart Model:")
flipkart_model = train_predict(flipkart_data)


   Year          Product Platform    Base_Price  Discount_Percentage  \
0  2013       Smartphone   Amazon  43708.610696            69.139027   
1  2013           Laptop   Amazon  95564.287577            24.523316   
2  2013       Television   Amazon  75879.454763            50.328133   
3  2013       Headphones   Amazon  63879.263578            55.697177   
4  2013  Washing Machine   Amazon  24041.677640            24.258253   

   Discounted_Price  
0      13488.902438  
1      72128.755073  
2      37690.741968  
3      28300.317128  
4      18209.586739  
Amazon Model:
Mean Absolute Error: 16.54664454277423

Flipkart Model:
Mean Absolute Error: 14.843303999863641


In [7]:
# Predict discounts for the next 10 years (2024-2033)
future_years = np.arange(2024, 2034)
future_products = np.tile(products, len(future_years))

# Create a DataFrame to store predictions
future_predictions = []

for product in products:
    for year in future_years:
        base_price = np.random.uniform(10000, 100000)
        amazon_discount = amazon_model.predict([[year, products.index(product), base_price]])[0]
        flipkart_discount = flipkart_model.predict([[year, products.index(product), base_price]])[0]
        
        amazon_price = base_price * (1 - amazon_discount / 100)
        flipkart_price = base_price * (1 - flipkart_discount / 100)
        
        better_platform = 'Amazon' if amazon_price < flipkart_price else 'Flipkart'
        
        future_predictions.append({
            'Year': year,
            'Product': product,
            'Base_Price': base_price,
            'Amazon_Discount': amazon_discount,
            'Amazon_Price': amazon_price,
            'Flipkart_Discount': flipkart_discount,
            'Flipkart_Price': flipkart_price,
            'Better_Platform': better_platform
        })

future_df = pd.DataFrame(future_predictions)
print(future_df.head())


c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarn

   Year     Product    Base_Price  Amazon_Discount  Amazon_Price  \
0  2024  Smartphone  81566.757529        36.532245  51768.590037   
1  2025  Smartphone  34374.902614        31.539698  23533.162080   
2  2026  Smartphone  49507.427864        40.586793  29413.950817   
3  2027  Smartphone  17061.074321        44.638759   9445.222543   
4  2028  Smartphone  12281.566907        52.591473   5822.509915   

   Flipkart_Discount  Flipkart_Price Better_Platform  
0          39.860961    49053.464086        Flipkart  
1          30.834455    23775.588739          Amazon  
2          33.159177    33091.172336          Amazon  
3          32.460805    11522.912218          Amazon  
4          31.868555     8367.608969          Amazon  


c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\SUDHARSAN M\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarn

In [10]:
# Display the recommended platform for each product and year
recommendations = future_df.groupby(['Year', 'Product'])['Better_Platform'].first().reset_index()
print(recommendations)


    Year          Product Better_Platform
0   2024       Headphones          Amazon
1   2024           Laptop          Amazon
2   2024     Refrigerator          Amazon
3   2024       Smartphone        Flipkart
4   2024       Television        Flipkart
..   ...              ...             ...
65  2033     Refrigerator          Amazon
66  2033       Smartphone          Amazon
67  2033       Television          Amazon
68  2033  Washing Machine          Amazon
69  2033            watch          Amazon

[70 rows x 3 columns]
